# Домашняя работа к первому уроку

### 1.Приведите по 2 примера, когда лучше максимизировать Precision, а когда Recall.

Максимизируем Precision
1. Мы хотим, чтобы на охраняемый обьект не проникли посторонние
2. Мы хотим обзванивать только тех, кто гарантированно согласится на наше предложение

Максимизируем Recall
1. Мы хотим, чтобы гарантированно все сотрудники нашей компании смогли проникнуть на охраняемый обьект
1. Мы хотим обзвонить максимальное количество клиентов, которым возможно интересно наше предложение

## 2.Почему мы используем F-меру, почему, например, нельзя просто взять среднее от Precision и Recall?

Если алгоритм отнесет все обьекты к одному классу,то Recall  будет 1, а Precision близким к нулю, но при среднемарифметическом
результат все-равно будет > 0,5. При F-мере, если один из показателей близок к 0, то и F-мера близка к нулю.

## 3.
Скачать данные из соревнования, пока можете работать только с train.csv. Считать их.

**Описание датасета**

* **Home Ownership** - домовладение
* **Annual Income** - годовой доход
* **Years in current job** - количество лет на текущем месте работы
* **Tax Liens** - налоговые обременения
* **Number of Open Accounts** - количество открытых счетов
* **Years of Credit History** - количество лет кредитной истории
* **Maximum Open Credit** - наибольший открытый кредит
* **Number of Credit Problems** - количество проблем с кредитом
* **Months since last delinquent** - количество месяцев с последней просрочки платежа
* **Bankruptcies** - банкротства
* **Purpose** - цель кредита
* **Term** - срок кредита
* **Current Loan Amount** - текущая сумма кредита
* **Current Credit Balance** - текущий кредитный баланс
* **Monthly Debt** - ежемесячный долг
* **Credit Score** - баллы кредитного рейтинга
* **Credit Default** - факт невыполнения кредитных обязательств (0 - погашен вовремя, 1 - просрочка)

In [5]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (roc_auc_score, roc_curve, auc, confusion_matrix, \
                             accuracy_score, classification_report, plot_confusion_matrix, \
                             plot_precision_recall_curve, precision_recall_curve, recall_score,
                             plot_roc_curve)

In [6]:
TRAIN_DATASET_PATH = 'data/course_project/train.csv'
# TEST_DATASET_PATH = './data/course_project/test.csv'

In [8]:
df_train = pd.read_csv(TRAIN_DATASET_PATH)
df_train.head(3)

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
0,Own Home,482087.0,NaN,0.0,11.0,26.3,685960.0,1.0,NaN,1.0,debt consolidation,Short Term,99999999.0,47386.0,7914.0,749.0,0
1,Own Home,1025487.0,10+ years,0.0,15.0,15.3,1181730.0,0.0,NaN,0.0,debt consolidation,Long Term,264968.0,394972.0,18373.0,737.0,1
2,Home Mortgage,751412.0,8 years,0.0,11.0,35.0,1182434.0,0.0,NaN,0.0,debt consolidation,Short Term,99999999.0,308389.0,13651.0,742.0,0


Посчитать основные статистики по каждому признаку (минимальные, максимальные, средние значения).
Сделать выводы об адекватности этих статистик.

In [10]:
df_train.describe(include = 'all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Home Ownership,7500,4,Home Mortgage,3637,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Annual Income,5943,NaN,NaN,NaN,1.36639e+06,845339,164597,844341,1.16839e+06,1.64014e+06,1.01493e+07
Years in current job,7129,11,10+ years,2332,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Tax Liens,7500,NaN,NaN,NaN,0.0301333,0.271604,0,0,0,0,7
Number of Open Accounts,7500,NaN,NaN,NaN,11.1309,4.90892,2,8,10,14,43
Years of Credit History,7500,NaN,NaN,NaN,18.3175,7.04195,4,13.5,17,21.8,57.7
Maximum Open Credit,7500,NaN,NaN,NaN,945154,1.60262e+07,0,279230,478159,793502,1.30473e+09
Number of Credit Problems,7500,NaN,NaN,NaN,0.17,0.498598,0,0,0,0,7
Months since last delinquent,3419,NaN,NaN,NaN,34.6926,21.6888,0,16,32,50,118
Bankruptcies,7486,NaN,NaN,NaN,0.117152,0.347192,0,0,0,0,4


### Посчитать для каждого признака, сколько в нем уникальных значений и как часто они встречаются.
Сделать выводы по каждому признаку.

In [16]:
for col in df_train.columns:
    colmn = df_train[col].value_counts()
    print (colmn)
    print('*'*100)

Home Mortgage    3637
Rent             3204
Own Home          647
Have Mortgage      12
Name: Home Ownership, dtype: int64
****************************************************************************************************
1161660.0    4
1058376.0    4
1338113.0    4
1043651.0    4
969475.0     4
            ..
1141254.0    1
7907382.0    1
1534478.0    1
972040.0     1
1654786.0    1
Name: Annual Income, Length: 5478, dtype: int64
****************************************************************************************************
10+ years    2332
2 years       705
3 years       620
< 1 year      563
5 years       516
1 year        504
4 years       469
6 years       426
7 years       396
8 years       339
9 years       259
Name: Years in current job, dtype: int64
****************************************************************************************************
0.0    7366
1.0      83
2.0      30
3.0      10
4.0       6
5.0       2
6.0       2
7.0       1
Name: Tax Liens, dtype: in

## 3. Используя выводы из двух предыдущих заданий, найдите те наблюдения, где есть пропуски, выбросы (неадекватные значения признака, которые отличаются от большинства), используя фильтрацию из pandas.

### Выглядит нездорово кредит на сумму 99999999. Скорее всего, ошибка

In [17]:
df_train.loc[df_train['Current Loan Amount']==99999999.0]

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
0,Own Home,482087.0,NaN,0.0,11.0,26.3,685960.0,1.0,NaN,1.0,debt consolidation,Short Term,99999999.0,47386.0,7914.0,749.0,0
2,Home Mortgage,751412.0,8 years,0.0,11.0,35.0,1182434.0,0.0,NaN,0.0,debt consolidation,Short Term,99999999.0,308389.0,13651.0,742.0,0
6,Home Mortgage,1511108.0,10+ years,0.0,9.0,20.3,388124.0,0.0,73.0,0.0,home improvements,Short Term,99999999.0,51623.0,2317.0,745.0,0
22,Rent,463790.0,NaN,0.0,6.0,28.3,288442.0,0.0,NaN,0.0,debt consolidation,Short Term,99999999.0,157700.0,5063.0,738.0,0
27,Rent,1004055.0,8 years,0.0,7.0,14.5,328394.0,0.0,NaN,0.0,debt consolidation,Short Term,99999999.0,195966.0,6041.0,734.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7441,Rent,1406836.0,1 year,0.0,12.0,22.6,2383260.0,0.0,NaN,0.0,debt consolidation,Short Term,99999999.0,461054.0,7386.0,747.0,0
7445,Home Mortgage,1564384.0,6 years,0.0,21.0,17.3,1545016.0,0.0,NaN,0.0,other,Long Term,99999999.0,1043442.0,32461.0,728.0,0
7464,Home Mortgage,1159152.0,6 years,0.0,14.0,22.0,389356.0,1.0,77.0,1.0,debt consolidation,Short Term,99999999.0,49438.0,18257.0,737.0,0
7475,Rent,848426.0,10+ years,0.0,8.0,20.1,526306.0,0.0,12.0,0.0,debt consolidation,Short Term,99999999.0,83638.0,16473.0,742.0,0


### Определим столбики с пропусками данных

In [28]:
for col in df_train.columns:
    count = df_train[col].count()
    if count < df_train.shape[0]:
        print(f'Column-{col} : q-ty = {count}')
#         print('*'*100)



Column-Annual Income : q-ty = 5943
Column-Years in current job : q-ty = 7129
Column-Months since last delinquent : q-ty = 3419
Column-Bankruptcies : q-ty = 7486
Column-Credit Score : q-ty = 5943


### Вызывыют удивление люди, у которых есть кредит, но указано, что им выдавалось максимально не более 0

In [37]:
df_train.loc[(df_train['Maximum Open Credit'] == 0) & (df_train['Current Loan Amount'] > 0)]

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
8,Home Mortgage,NaN,5 years,0.0,17.0,15.7,0.0,1.0,NaN,1.0,home improvements,Short Term,129734.0,19.0,17.0,NaN,0
91,Rent,601369.0,10+ years,0.0,6.0,16.9,0.0,0.0,14.0,0.0,debt consolidation,Short Term,165924.0,0.0,9021.0,738.0,0
196,Rent,815157.0,7 years,0.0,3.0,8.8,0.0,0.0,NaN,0.0,small business,Short Term,218042.0,0.0,5855.0,738.0,0
294,Home Mortgage,2033931.0,7 years,0.0,6.0,21.1,0.0,0.0,NaN,0.0,home improvements,Short Term,108042.0,0.0,30340.0,742.0,0
309,Rent,1245792.0,3 years,0.0,5.0,22.4,0.0,0.0,58.0,0.0,major purchase,Long Term,338096.0,0.0,395.0,659.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6593,Home Mortgage,NaN,8 years,0.0,10.0,13.0,0.0,0.0,NaN,0.0,debt consolidation,Long Term,672078.0,19.0,8860.0,NaN,0
6851,Home Mortgage,2251728.0,10+ years,0.0,8.0,15.4,0.0,0.0,NaN,0.0,home improvements,Long Term,760452.0,0.0,36778.0,725.0,0
7014,Home Mortgage,1223771.0,5 years,0.0,9.0,24.0,0.0,0.0,NaN,0.0,small business,Long Term,751454.0,0.0,15348.0,737.0,1
7124,Rent,NaN,1 year,0.0,4.0,11.0,0.0,0.0,50.0,0.0,debt consolidation,Short Term,134024.0,0.0,0.0,NaN,0


### Проверим тех, кто должен платить, но при этом у него задолженность 0

In [33]:
df_train.loc[(df_train['Current Credit Balance'] == 0) & (df_train['Monthly Debt'] > 0) ]

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
91,Rent,601369.0,10+ years,0.0,6.0,16.9,0.0,0.0,14.0,0.0,debt consolidation,Short Term,165924.0,0.0,9021.0,738.0,0
196,Rent,815157.0,7 years,0.0,3.0,8.8,0.0,0.0,NaN,0.0,small business,Short Term,218042.0,0.0,5855.0,738.0,0
294,Home Mortgage,2033931.0,7 years,0.0,6.0,21.1,0.0,0.0,NaN,0.0,home improvements,Short Term,108042.0,0.0,30340.0,742.0,0
309,Rent,1245792.0,3 years,0.0,5.0,22.4,0.0,0.0,58.0,0.0,major purchase,Long Term,338096.0,0.0,395.0,659.0,0
319,Own Home,NaN,< 1 year,0.0,2.0,10.5,0.0,0.0,NaN,0.0,business loan,Short Term,76670.0,0.0,6432.0,NaN,0
544,Home Mortgage,NaN,2 years,0.0,15.0,22.5,0.0,0.0,NaN,0.0,other,Short Term,396594.0,0.0,22556.0,NaN,0
585,Home Mortgage,4772800.0,8 years,0.0,9.0,35.5,0.0,0.0,NaN,0.0,small business,Short Term,99999999.0,0.0,2227.0,747.0,0
1163,Home Mortgage,4523064.0,3 years,0.0,6.0,14.2,0.0,0.0,38.0,0.0,home improvements,Long Term,392788.0,0.0,19902.0,690.0,0
1338,Rent,NaN,2 years,0.0,8.0,17.6,0.0,0.0,43.0,0.0,debt consolidation,Short Term,22176.0,0.0,12677.0,NaN,0
1404,Rent,349391.0,< 1 year,0.0,4.0,12.7,0.0,0.0,0.0,NaN,other,Short Term,11242.0,0.0,1243.0,747.0,0


### Проверим, есть ли те, кто не платит ежемесячно, но имеет кредит

In [36]:
df_train.loc[(df_train['Monthly Debt'] == 0) &(df_train['Current Credit Balance'] > 0)]

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default


### Кажется, что если текущий кредит больше предыдущих, он должен стоят в столбе максимально выданный кредит

In [41]:
df_train.loc[df_train['Maximum Open Credit'] < df_train['Current Loan Amount']]

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
0,Own Home,482087.0,NaN,0.0,11.0,26.3,685960.0,1.0,NaN,1.0,debt consolidation,Short Term,99999999.0,47386.0,7914.0,749.0,0
2,Home Mortgage,751412.0,8 years,0.0,11.0,35.0,1182434.0,0.0,NaN,0.0,debt consolidation,Short Term,99999999.0,308389.0,13651.0,742.0,0
6,Home Mortgage,1511108.0,10+ years,0.0,9.0,20.3,388124.0,0.0,73.0,0.0,home improvements,Short Term,99999999.0,51623.0,2317.0,745.0,0
8,Home Mortgage,NaN,5 years,0.0,17.0,15.7,0.0,1.0,NaN,1.0,home improvements,Short Term,129734.0,19.0,17.0,NaN,0
9,Home Mortgage,NaN,1 year,0.0,10.0,24.6,511302.0,0.0,6.0,0.0,debt consolidation,Long Term,572880.0,205333.0,17613.0,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7487,Home Mortgage,1542952.0,10+ years,0.0,7.0,12.5,232364.0,0.0,59.0,0.0,debt consolidation,Long Term,267982.0,198873.0,15430.0,695.0,1
7488,Rent,465405.0,5 years,0.0,5.0,7.7,51744.0,0.0,33.0,0.0,home improvements,Short Term,129338.0,8721.0,13535.0,728.0,0
7491,Home Mortgage,2833185.0,6 years,0.0,18.0,21.3,280170.0,0.0,6.0,0.0,debt consolidation,Short Term,437404.0,108889.0,61150.0,704.0,0
7495,Rent,402192.0,< 1 year,0.0,3.0,8.5,107866.0,0.0,NaN,0.0,other,Short Term,129360.0,73492.0,1900.0,697.0,0


### Проверим сколько месяцев будут выплачивать полученные кредиты

In [50]:
df_train['Number_month_credit'] = df_train['Current Credit Balance'] / df_train['Monthly Debt']
df_train['Number_month_credit'].value_counts()

0.000000     47
6.031599      1
26.976538     1
6.992308      1
8.150248      1
             ..
3.258758      1
33.867967     1
13.961009     1
11.869060     1
42.682641     1
Name: Number_month_credit, Length: 7448, dtype: int64

In [51]:
df_train['Number_month_credit'].describe()

count    7494.000000
mean       17.901800
std        16.873370
min         0.000000
25%         8.288474
50%        14.338532
75%        23.499093
max       502.307475
Name: Number_month_credit, dtype: float64

### Есть ли те, у кого кредит на 0 месяцев, но существует долг

In [59]:
df_train.loc[(df_train['Number_month_credit'] == 0) & (df_train['Current Credit Balance'] > 0)]

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default,Number_month_credit


### Есть ли люди, чей кредит больше их жизни (80 лет)?

In [54]:
df_train.loc[(df_train['Number_month_credit']/12 + df_train['Years of Credit History']+ 20) > 80]

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default,Number_month_credit
2613,Home Mortgage,685596.0,6 years,0.0,13.0,23.0,1111506.0,0.0,NaN,0.0,debt consolidation,Long Term,136906.0,356136.0,709.0,737.0,0,502.307475
6505,Home Mortgage,NaN,NaN,0.0,6.0,42.4,2748240.0,0.0,11.0,0.0,debt consolidation,Short Term,106260.0,1571243.0,4753.0,NaN,1,330.579213


### Есть ли те у кого доходов не хватает для погашения кредита ?

In [57]:
df_train.loc[(df_train['Monthly Debt']*12) > (df_train['Annual Income']*0.7)]

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default,Number_month_credit
